# Preliminaries

In [1]:
import os
import psycopg2
import pandas as pd

In [2]:
credentials = os.getenv('MDI_POSTGRESQL')
sql_connection = psycopg2.connect(credentials)
cur = sql_connection.cursor()

# Preview of tables

In [3]:
employees = pd.read_sql('SELECT * FROM employees;', sql_connection)
employees

,id,name,age,salary,bonus,city,dep_id
0,1,Monika,25,1000,NaN,Palanga,1
1,2,Aidas,51,900,300.0,Vilnius,2
2,3,Algimantas,34,1500,NaN,Vilnius,2
3,4,Julius,42,2300,250.0,Vilnius,3
4,5,Vaidas,33,1700,NaN,Palanga,3


In [4]:
departments = pd.read_sql("SELECT * FROM departments;", sql_connection)
departments

,id,name,location,budget_last_year,budget_this_year,mgr_id
0,1,HR,Palanga,10.0,12,1.0
1,2,Sales,Vilnius,20.0,25,2.0
2,3,BI,Vilnius,NaN,7,5.0
3,4,R+D,Vilnius,5.0,10,NaN


# Solutions

## Query 1

**Task:** % of departments without managers

**Solution**:

In [5]:
pd.read_sql(
    """
    SELECT
        COUNT(*) * 100.0 / (
            SELECT COUNT(*) FROM departments
        ) AS "%_departments_no_managers"
    FROM
        departments
    WHERE
        Mgr_id IS NULL
    GROUP BY
        Mgr_id;
    """,
    sql_connection,
)

,%_departments_no_managers
0,25.0


## Query 2

**Task:** Department with the highest increase of budget from last year to this year, and budget amount 
increase.

**Solution**:

In [6]:
pd.read_sql(
    """
    SELECT
        Name AS "Department_Name",
        ABS(
            Budget_this_year - COALESCE(Budget_last_year, 0)
        ) AS "Amount"
    FROM
        departments
    ORDER BY
        "Amount" DESC
    LIMIT
        1;
    """,
    sql_connection
)

,Department_Name,Amount
0,BI,7


## Query 3


**Task:** For each employee, the name of department they manage

**Solution**:

In [7]:
pd.read_sql(
    """
    SELECT
        employees.Name AS "Employee_Name",
        departments.Name AS "Department"
    FROM
        employees
        LEFT JOIN departments ON employees.ID = departments.Mgr_id;
    """,
    sql_connection
)

,Employee_Name,Department
0,Monika,HR
1,Aidas,Sales
2,Vaidas,BI
3,Julius,None
4,Algimantas,None


## Query 4

**Task:** Name of each manager and average salary in they department

**Solution**:

In [8]:
pd.read_sql(
    """
    SELECT
        Manager,
        Amount
    FROM (
        SELECT
        employees.Dep_id,
            employees.Name AS Manager
        FROM
            employees
            INNER JOIN departments ON employees.ID = departments.Mgr_id
        ) table1
    INNER JOIN (
        SELECT
            employees.Dep_id,
            AVG(employees.Salary) AS Amount
        FROM
            employees
        GROUP BY 
            employees.Dep_id
        ) table2 ON table1.dep_id = table2.dep_id
    ORDER BY
        Amount;
    """,
    sql_connection)

,manager,amount
0,Monika,1000.0
1,Aidas,1200.0
2,Vaidas,2000.0


## Query 5

**Task:** For the department with highest budget this year, name of the manager, budget and 
department they manage. 

**Solution**:

In [9]:
pd.read_sql(
    """
    SELECT
        employees.Name AS "Manager Name",
        departments.Name AS "Department Name",
        departments.Budget_this_year AS "Budget"
    FROM
        employees
        INNER JOIN departments ON employees.ID = departments.Mgr_id
    ORDER BY
        "Budget" DESC
    LIMIT
        1;
    """,
    sql_connection
)

,Manager Name,Department Name,Budget
0,Aidas,Sales,25


## Query 6

**Task:** Name of departments spending less than 2000 in salaries

**Solution**:

In [10]:
pd.read_sql(
    """
    SELECT
        departments.Name AS "Department Name"
    FROM
        departments
        LEFT JOIN employees ON employees.Dep_id = departments.ID
    GROUP BY
        departments.Name
    HAVING (
        SUM(employees.Salary) IS NULL
        OR SUM(employees.Salary) < 2000
        )
    ORDER BY
        "Department Name" ASC;
    """,
    sql_connection
)

,Department Name
0,HR
1,R+D


## Query 7

**Task:** For each manager, employee id and location of the department they manage

**Solution**:

In [11]:
pd.read_sql(
    """
    SELECT
        employees.ID AS "Employee_id",
        departments.Location
    FROM
        departments
        INNER JOIN employees ON employees.ID = departments.Mgr_id;
    """,
    sql_connection
)

,Employee_id,location
0,1,Palanga
1,2,Vilnius
2,5,Vilnius


## Query 8

**Task:** Number of departments managed by an employee belonging to another department 

**Solution**:

In [12]:
pd.read_sql(
    """
    SELECT
        COUNT(*) AS "# departments"
    FROM
        employees
        INNER JOIN departments ON employees.ID = departments.Mgr_id
    WHERE
        employees.Dep_id != departments.ID;
    """,
    sql_connection
)

,# departments
0,0


In [13]:
sql_connection.close()